In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

for dirname, _, filenames in os.walk("/kaggle/input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/google-quest-challenge/sample_submission.csv
/kaggle/input/google-quest-challenge/train.csv
/kaggle/input/google-quest-challenge/test.csv
/kaggle/input/bert-base-uncased-offline/pytorch/default/1/bert-base-uncased-offline/config.json
/kaggle/input/bert-base-uncased-offline/pytorch/default/1/bert-base-uncased-offline/tokenizer_config.json
/kaggle/input/bert-base-uncased-offline/pytorch/default/1/bert-base-uncased-offline/model.safetensors
/kaggle/input/bert-base-uncased-offline/pytorch/default/1/bert-base-uncased-offline/special_tokens_map.json
/kaggle/input/bert-base-uncased-offline/pytorch/default/1/bert-base-uncased-offline/vocab.txt
/kaggle/input/lab1_model/pytorch/default/1/quest_model.bin


In [2]:
offline = True

# install and import library

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from transformers import (
    BertConfig,
    BertModel,
    BertTokenizer,
    get_linear_schedule_with_warmup,
)

2025-06-18 16:30:04.651796: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750264204.794872      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750264204.840108      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# import data

In [ ]:
df = pd.read_csv("/kaggle/input/google-quest-challenge/train.csv")

In [5]:
df.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skin...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,russellpierce,https://rpg.stackexchange.com/users/8774,It might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,...,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Joe Baker,https://electronics.stackexchange.com/users/10157,Do you even need grooves? We make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,...,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...",Scimonster,https://judaism.stackexchange.com/users/5151,"Sending an ""affidavit"" it is a dispute between...",Y e z,https://judaism.stackexchange.com/users/4794,http://judaism.stackexchange.com/questions/551...,CULTURE,...,0.888889,0.833333,0.333333,0.833333,1.000000,0.800000,0.0,0.000000,1.000000,1.000000
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,leigero,https://graphicdesign.stackexchange.com/users/...,Check out Image Trace in Adobe Illustrator. \n...,q2ra,https://graphicdesign.stackexchange.com/users/...,http://graphicdesign.stackexchange.com/questio...,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,1.000000,1.000000


In [ ]:
TARGET_COLUMNS = [
    "question_asker_intent_understanding",
    "question_body_critical",
    "question_conversational",
    "question_expect_short_answer",
    "question_fact_seeking",
    "question_has_commonly_accepted_answer",
    "question_interestingness_others",
    "question_interestingness_self",
    "question_multi_intent",
    "question_not_really_a_question",
    "question_opinion_seeking",
    "question_type_choice",
    "question_type_compare",
    "question_type_consequence",
    "question_type_definition",
    "question_type_entity",
    "question_type_instructions",
    "question_type_procedure",
    "question_type_reason_explanation",
    "question_type_spelling",
    "question_well_written",
    "answer_helpful",
    "answer_level_of_information",
    "answer_plausible",
    "answer_relevance",
    "answer_satisfaction",
    "answer_type_instructions",
    "answer_type_procedure",
    "answer_type_reason_explanation",
    "answer_well_written",
]


In [7]:
TARGET_COLUMNS

['question_asker_intent_understanding',
 'question_body_critical',
 'question_conversational',
 'question_expect_short_answer',
 'question_fact_seeking',
 'question_has_commonly_accepted_answer',
 'question_interestingness_others',
 'question_interestingness_self',
 'question_multi_intent',
 'question_not_really_a_question',
 'question_opinion_seeking',
 'question_type_choice',
 'question_type_compare',
 'question_type_consequence',
 'question_type_definition',
 'question_type_entity',
 'question_type_instructions',
 'question_type_procedure',
 'question_type_reason_explanation',
 'question_type_spelling',
 'question_well_written',
 'answer_helpful',
 'answer_level_of_information',
 'answer_plausible',
 'answer_relevance',
 'answer_satisfaction',
 'answer_type_instructions',
 'answer_type_procedure',
 'answer_type_reason_explanation',
 'answer_well_written']

In [ ]:
class QuestDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=256):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text = row["question_title"] + " " + row["question_body"] + " " + row["answer"]
        inputs = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        item = {key: val.squeeze(0) for key, val in inputs.items()}
        item["labels"] = torch.tensor(
            row[TARGET_COLUMNS].values.astype(float), dtype=torch.float
        )
        return item

# Tokenizer & DataLoader


In [ ]:
if not offline:
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    tokenizer.save_pretrained("bert-base-uncased-offline")
else:
    tokenizer = BertTokenizer.from_pretrained(
        "/kaggle/input/bert-base-uncased-offline/pytorch/default/1/bert-base-uncased-offline",
        local_files_only=True,
    )

train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
train_dataset = QuestDataset(train_df, tokenizer)
val_dataset = QuestDataset(val_df, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)


# Model

In [ ]:
class QuestModel(nn.Module):
    def __init__(self, num_labels):
        super().__init__()
        if not offline:
            self.bert = BertModel.from_pretrained("bert-base-uncased")
            self.bert.save_pretrained("bert-base-uncased-offline")
        else:
            self.bert = BertModel.from_pretrained(
                "/kaggle/input/bert-base-uncased-offline/pytorch/default/1/bert-base-uncased-offline",
                local_files_only=True,
            )
        self.dropout = nn.Dropout(0.2)
        self.out = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids, labels=None):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        pooled_output = self.dropout(outputs.pooler_output)
        logits = self.out(pooled_output)
        if labels is not None:
            loss_fn = nn.BCEWithLogitsLoss()
            loss = loss_fn(logits, labels)
            return loss, logits
        return logits


model = QuestModel(num_labels=len(TARGET_COLUMNS)).cuda()


# Optimizer & Scheduler


In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 3
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=total_steps
)

# Training

In [ ]:
if not offline:
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader):
            optimizer.zero_grad()
            input_ids = batch["input_ids"].cuda()
            attention_mask = batch["attention_mask"].cuda()
            token_type_ids = batch["token_type_ids"].cuda()
            labels = batch["labels"].cuda()
            loss, _ = model(input_ids, attention_mask, token_type_ids, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1} Loss: {total_loss / len(train_loader)}")

    torch.save(model.state_dict(), "quest_model.bin")

else:
    model.load_state_dict(
        torch.load(
            "/kaggle/input/lab1_model/pytorch/default/1/quest_model.bin",
            map_location="cpu",
        )
    )

model.eval()
val_loss = 0
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch["input_ids"].cuda()
        attention_mask = batch["attention_mask"].cuda()
        token_type_ids = batch["token_type_ids"].cuda()
        labels = batch["labels"].cuda()
        loss, _ = model(input_ids, attention_mask, token_type_ids, labels)
        val_loss += loss.item()
print(f"Validation Loss: {val_loss / len(val_loader)}")


Validation Loss: 0.3738970689867672


# Submission 

In [ ]:
test_df = pd.read_csv("/kaggle/input/google-quest-challenge/test.csv")


class QuestTestDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=256):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text = (
            str(row["question_title"])
            + " "
            + str(row["question_body"])
            + " "
            + str(row["answer"])
        )
        inputs = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        item = {key: val.squeeze(0) for key, val in inputs.items()}
        return item

In [14]:
test_dataset = QuestTestDataset(test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

model = QuestModel(num_labels=len(TARGET_COLUMNS))

CHECKPOINT_PATH = '/kaggle/input/lab1_model/pytorch/default/1/quest_model.bin' if offline else '/kaggle/working/quest_model.bin'
model.load_state_dict(torch.load(CHECKPOINT_PATH, map_location='cpu'))

model.eval()
model.to('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
all_preds = []
device = "cuda" if torch.cuda.is_available() else "cpu"
with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        logits = model(input_ids, attention_mask, token_type_ids)
        preds = torch.sigmoid(logits).cpu().numpy()
        all_preds.append(preds)
all_preds = np.concatenate(all_preds, axis=0)

100%|██████████| 30/30 [00:07<00:00,  3.75it/s]


In [ ]:
submission = pd.read_csv("/kaggle/input/google-quest-challenge/sample_submission.csv")
submission[TARGET_COLUMNS] = all_preds
submission.to_csv("submission.csv", index=False)
print("submission.csv created!")

submission.csv created!
